In [1]:
import roberta as my_roberta
from transformers.models.roberta import modeling_roberta as hf_roberta

import torch
import haliax as hax
import jax.random as jrandom
import numpy as np

c:\Users\julie\anaconda3\envs\levanter2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-30 10:42:02,733	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
from transformers import AutoConfig

hf_model_str = "FacebookAI/roberta-base"

hf_config = AutoConfig.from_pretrained(hf_model_str)
hf_config.hidden_dropout_prob = 0
hf_config.attention_probs_dropout_prob = 0
my_config = my_roberta.RobertaConfig.from_hf_config(hf_config)

key = jrandom.PRNGKey(0)

In [3]:
EmbedAtt = my_config.EmbedAtt
Embed = my_config.Embed
Mlp = my_config.Mlp
Pos = my_config.Pos
KeyPos = my_config.KeyPos
Batch = hax.Axis("batch", 2)

x_embed = hax.random.normal(key, (Batch, Pos, Embed))
x_embed_att = hax.random.normal(key, (Batch, Pos, EmbedAtt))
x_mlp = hax.random.normal(key, (Batch, Pos, Mlp))
# x = x[{"position": slice(Pos.size-2)}]

x_embed_torch = torch.from_numpy(np.array(x_embed.array))
x_embed_att_torch = torch.from_numpy(np.array(x_embed_att.array))
x_mlp_torch = torch.from_numpy(np.array(x_mlp.array))

mask = hax.ones((Batch, Pos))[{"position": slice(0,-2)}]
mask_torch = torch.from_numpy(np.array(mask.array))
# mask_torch = torch.ones((2, hf_config.num_attention_heads, hf_config.max_position_embeddings -2, hf_config.max_position_embeddings -2))

Vocab = hax.Axis("vocab", my_config.vocab_size)

input_embeds = hax.random.uniform(key, (Batch, Pos, Embed))
input_embeds = input_embeds[{"position": slice(0,-2)}]

input_ids = hax.random.randint(key, (Batch, Pos), minval = 0, maxval = my_config.vocab_size)
input_ids = input_ids[{"position": slice(0,-2)}]

input_embeds_torch = torch.from_numpy(np.array(input_embeds.array))
input_ids_torch = torch.from_numpy(np.array(input_ids.array))

In [4]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(hf_model_str)

In [5]:
def check(my_output, hf_output, p=False, pp=False, ppp=True, precision=1e-4):
    
    print(my_output.shape)
    print(hf_output.shape)

    success = np.isclose(hf_output, np.array(my_output), rtol=precision, atol=precision).all()

    if success:
        print("Success!!!")
    else:
        print("Fail :((((")
    
    if ppp:
        acc_prev = None
        for i in range(15):
            prec = 10 ** (-1*i)
            acc = np.isclose(hf_output, np.array(my_output), rtol=precision, atol=prec).mean()
            if acc != acc_prev:
                print(f"Iteration {i}, Precision {prec}:\t{acc}")
            acc_prev = acc

    if p:   
        acc = np.isclose(hf_output, np.array(my_output), rtol=precision, atol=precision).mean()
        print(f"Accuracy: {acc}")
        print(f"Jax:\n{torch.tensor(np.array(my_output))}\nTorch:\n{hf_output}")

    if pp:
        diff = torch.tensor(np.array(my_output)) - hf_output
        print(f"Mean: {diff.abs().mean()}")
        print(f"Stdev: {diff.std()}")
        print(f"Difference:\n{diff}")

In [6]:
'''# Testing RobertaSelfOutput

my_self_output = my_roberta.RobertaSelfOutput.init(my_config, key=key)
state = my_self_output.to_state_dict()

state = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}

print(state.keys())

hf_self_output = hf_roberta.RobertaSelfOutput(hf_config)
hf_self_output.load_state_dict(state, strict=True)

my_output = my_self_output(x_embed_att, x_embed, key=key)
hf_output = hf_self_output(x_embed_att_torch, x_embed_torch)

check(my_output.array, hf_output.detach(), ppp=True)'''

'# Testing RobertaSelfOutput\n\nmy_self_output = my_roberta.RobertaSelfOutput.init(my_config, key=key)\nstate = my_self_output.to_state_dict()\n\nstate = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}\n\nprint(state.keys())\n\nhf_self_output = hf_roberta.RobertaSelfOutput(hf_config)\nhf_self_output.load_state_dict(state, strict=True)\n\nmy_output = my_self_output(x_embed_att, x_embed, key=key)\nhf_output = hf_self_output(x_embed_att_torch, x_embed_torch)\n\ncheck(my_output.array, hf_output.detach(), ppp=True)'

In [7]:
'''# Testing RobertaSelfAttention

my_attn_output = my_roberta.RobertaSelfAttention.init(my_config, key=key)
state = my_attn_output.to_state_dict()

state = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}

print(state.keys())

hf_attn_output  = hf_roberta.RobertaSelfAttention(hf_config)
hf_attn_output.load_state_dict(state, strict=True)

my_output = my_attn_output(x_embed, mask, key=key)
hf_output = hf_attn_output(x_embed_torch, mask_torch)

check(my_output.array, hf_output[0].detach(), ppp=True)'''

'# Testing RobertaSelfAttention\n\nmy_attn_output = my_roberta.RobertaSelfAttention.init(my_config, key=key)\nstate = my_attn_output.to_state_dict()\n\nstate = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}\n\nprint(state.keys())\n\nhf_attn_output  = hf_roberta.RobertaSelfAttention(hf_config)\nhf_attn_output.load_state_dict(state, strict=True)\n\nmy_output = my_attn_output(x_embed, mask, key=key)\nhf_output = hf_attn_output(x_embed_torch, mask_torch)\n\ncheck(my_output.array, hf_output[0].detach(), ppp=True)'

In [8]:
'''# Testing RobertaAttention

my_func = my_roberta.RobertaAttention.init(my_config, key=key)
state = my_func.to_state_dict()

state = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}

print(state.keys())

hf_func = hf_roberta.RobertaAttention(hf_config)
hf_func.load_state_dict(state, strict=True)

my_output = my_func(hidden_states=x_embed, attention_mask=mask, key=key)
hf_output = hf_func(hidden_states=x_embed_torch, attention_mask=mask_torch)

check(my_output.array, hf_output[0].detach(), ppp=True)'''

'# Testing RobertaAttention\n\nmy_func = my_roberta.RobertaAttention.init(my_config, key=key)\nstate = my_func.to_state_dict()\n\nstate = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}\n\nprint(state.keys())\n\nhf_func = hf_roberta.RobertaAttention(hf_config)\nhf_func.load_state_dict(state, strict=True)\n\nmy_output = my_func(hidden_states=x_embed, attention_mask=mask, key=key)\nhf_output = hf_func(hidden_states=x_embed_torch, attention_mask=mask_torch)\n\ncheck(my_output.array, hf_output[0].detach(), ppp=True)'

In [9]:
'''# Testing RobertaIntermediate

my_int_output = my_roberta.RobertaIntermediate.init(my_config, key=key)
state = my_int_output.to_state_dict()

state = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}

print(state.keys())

hf_int_output = hf_roberta.RobertaIntermediate(hf_config)
hf_int_output.load_state_dict(state, strict=True)

my_output = my_int_output(x_embed, key=key)
hf_output = hf_int_output(x_embed_torch)

check(my_output.array, hf_output.detach(), ppp=True)'''

'# Testing RobertaIntermediate\n\nmy_int_output = my_roberta.RobertaIntermediate.init(my_config, key=key)\nstate = my_int_output.to_state_dict()\n\nstate = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}\n\nprint(state.keys())\n\nhf_int_output = hf_roberta.RobertaIntermediate(hf_config)\nhf_int_output.load_state_dict(state, strict=True)\n\nmy_output = my_int_output(x_embed, key=key)\nhf_output = hf_int_output(x_embed_torch)\n\ncheck(my_output.array, hf_output.detach(), ppp=True)'

In [10]:
'''# Testing RobertaOutput

my_int_output = my_roberta.RobertaOutput.init(my_config, key=key)
state = my_int_output.to_state_dict()

state = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}

print(state.keys())

hf_int_output = hf_roberta.RobertaOutput(hf_config)
hf_int_output.load_state_dict(state, strict=True)

my_output = my_int_output(x_mlp, x_embed, key=key)
hf_output = hf_int_output(x_mlp_torch, x_embed_torch)

check(my_output.array, hf_output.detach(), ppp=True)'''

'# Testing RobertaOutput\n\nmy_int_output = my_roberta.RobertaOutput.init(my_config, key=key)\nstate = my_int_output.to_state_dict()\n\nstate = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}\n\nprint(state.keys())\n\nhf_int_output = hf_roberta.RobertaOutput(hf_config)\nhf_int_output.load_state_dict(state, strict=True)\n\nmy_output = my_int_output(x_mlp, x_embed, key=key)\nhf_output = hf_int_output(x_mlp_torch, x_embed_torch)\n\ncheck(my_output.array, hf_output.detach(), ppp=True)'

In [11]:
'''# Testing RobertaLayer

my_func = my_roberta.RobertaLayer.init(my_config, key=key)
state = my_func.to_state_dict()

state = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}

print(state.keys())

hf_func = hf_roberta.RobertaLayer(hf_config)
hf_func.load_state_dict(state, strict=True)

my_output = my_func(hidden_states=x_embed, attention_mask=mask, key=key)
hf_output = hf_func(hidden_states=x_embed_torch, attention_mask=mask_torch)

check(my_output.array, hf_output[0].detach(), False, False, True, 1e-4)'''

'# Testing RobertaLayer\n\nmy_func = my_roberta.RobertaLayer.init(my_config, key=key)\nstate = my_func.to_state_dict()\n\nstate = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}\n\nprint(state.keys())\n\nhf_func = hf_roberta.RobertaLayer(hf_config)\nhf_func.load_state_dict(state, strict=True)\n\nmy_output = my_func(hidden_states=x_embed, attention_mask=mask, key=key)\nhf_output = hf_func(hidden_states=x_embed_torch, attention_mask=mask_torch)\n\ncheck(my_output.array, hf_output[0].detach(), False, False, True, 1e-4)'

In [12]:
'''# Testing RobertaEncoder

my_func = my_roberta.RobertaEncoder.init(my_config, key=key)
state = my_func.to_state_dict()

state = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}

print(state.keys())

hf_func = hf_roberta.RobertaEncoder(hf_config)
hf_func.load_state_dict(state, strict=True)

my_output = my_func(hidden_states=x_embed, attention_mask=mask, key=key)
hf_output = hf_func(hidden_states=x_embed_torch, attention_mask=mask_torch)

check(my_output.array, hf_output[0].detach(), False, False, True, 1e-4)'''

'# Testing RobertaEncoder\n\nmy_func = my_roberta.RobertaEncoder.init(my_config, key=key)\nstate = my_func.to_state_dict()\n\nstate = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}\n\nprint(state.keys())\n\nhf_func = hf_roberta.RobertaEncoder(hf_config)\nhf_func.load_state_dict(state, strict=True)\n\nmy_output = my_func(hidden_states=x_embed, attention_mask=mask, key=key)\nhf_output = hf_func(hidden_states=x_embed_torch, attention_mask=mask_torch)\n\ncheck(my_output.array, hf_output[0].detach(), False, False, True, 1e-4)'

In [13]:
'''# Testing RobertaEmbedding

Vocab = hax.Axis("vocab", my_config.vocab_size)

input_embeds = hax.random.uniform(key, (Batch, Pos, Embed))
input_embeds = input_embeds[{"position": slice(0,-2)}]
input_ids = hax.random.randint(key, (Batch, Pos), minval = 0, maxval = my_config.vocab_size)
input_ids = input_ids[{"position": slice(0,-2)}]

input_embeds_torch = torch.from_numpy(np.array(input_embeds.array))
input_ids_torch = torch.from_numpy(np.array(input_ids.array))

my_func = my_roberta.RobertaEmbedding.init(Vocab, my_config, key=key)
state = my_func.to_state_dict()

state = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}

print(state.keys())

hf_func = hf_roberta.RobertaEmbeddings(hf_config)
hf_func.load_state_dict(state, strict=True)

# my_output = my_func.embed(input_embeds=input_embeds, key=key)
# hf_output = hf_func(inputs_embeds=input_embeds_torch)

my_output = my_func.embed(input_ids=input_ids, key=key)
hf_output = hf_func(input_ids=input_ids_torch)

check(my_output.array, hf_output.detach())'''

'# Testing RobertaEmbedding\n\nVocab = hax.Axis("vocab", my_config.vocab_size)\n\ninput_embeds = hax.random.uniform(key, (Batch, Pos, Embed))\ninput_embeds = input_embeds[{"position": slice(0,-2)}]\ninput_ids = hax.random.randint(key, (Batch, Pos), minval = 0, maxval = my_config.vocab_size)\ninput_ids = input_ids[{"position": slice(0,-2)}]\n\ninput_embeds_torch = torch.from_numpy(np.array(input_embeds.array))\ninput_ids_torch = torch.from_numpy(np.array(input_ids.array))\n\nmy_func = my_roberta.RobertaEmbedding.init(Vocab, my_config, key=key)\nstate = my_func.to_state_dict()\n\nstate = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}\n\nprint(state.keys())\n\nhf_func = hf_roberta.RobertaEmbeddings(hf_config)\nhf_func.load_state_dict(state, strict=True)\n\n# my_output = my_func.embed(input_embeds=input_embeds, key=key)\n# hf_output = hf_func(inputs_embeds=input_embeds_torch)\n\nmy_output = my_func.embed(input_ids=input_ids, key=key)\nhf_output = hf_func(input_ids=input_ids_t

In [14]:
'''# Testing RobertaPooler

my_pool = my_roberta.RobertaPooler.init(my_config, key=key)
state = my_pool.to_state_dict()

state = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}

print(state.keys())

hf_pool = hf_roberta.RobertaPooler(hf_config)
hf_pool.load_state_dict(state, strict=True)

my_output = my_pool(x_embed, key=key)
hf_output = hf_pool(x_embed_torch)

check(my_output.array, hf_output.detach(), ppp=True)'''

'# Testing RobertaPooler\n\nmy_pool = my_roberta.RobertaPooler.init(my_config, key=key)\nstate = my_pool.to_state_dict()\n\nstate = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}\n\nprint(state.keys())\n\nhf_pool = hf_roberta.RobertaPooler(hf_config)\nhf_pool.load_state_dict(state, strict=True)\n\nmy_output = my_pool(x_embed, key=key)\nhf_output = hf_pool(x_embed_torch)\n\ncheck(my_output.array, hf_output.detach(), ppp=True)'

In [15]:
'''# Testing RobertaModel

my_pool = my_roberta.RobertaModel.init(Vocab, my_config, add_pooling_layer=True, key=key)
state = my_pool.to_state_dict()

state = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}

print(state.keys())

hf_pool = hf_roberta.RobertaModel(hf_config, add_pooling_layer=True)
hf_pool.load_state_dict(state, strict=True)

my_output = my_pool(input_ids = input_ids, attention_mask=mask, key=key)
hf_output = hf_pool(input_ids = input_ids_torch, attention_mask=mask_torch, return_dict=False)

check(my_output[0].array, hf_output[0].detach(), ppp=True)'''

'# Testing RobertaModel\n\nmy_pool = my_roberta.RobertaModel.init(Vocab, my_config, add_pooling_layer=True, key=key)\nstate = my_pool.to_state_dict()\n\nstate = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}\n\nprint(state.keys())\n\nhf_pool = hf_roberta.RobertaModel(hf_config, add_pooling_layer=True)\nhf_pool.load_state_dict(state, strict=True)\n\nmy_output = my_pool(input_ids = input_ids, attention_mask=mask, key=key)\nhf_output = hf_pool(input_ids = input_ids_torch, attention_mask=mask_torch, return_dict=False)\n\ncheck(my_output[0].array, hf_output[0].detach(), ppp=True)'

In [16]:
# Testing RobertaForMaskedLM

my_pool = my_roberta.RobertaForMaskedLM.init(Vocab, my_config, key=key)
state = my_pool.to_state_dict()

state = {k: torch.from_numpy(np.array(v)) for k, v in state.items()}

print(state.keys())

hf_pool = hf_roberta.RobertaForMaskedLM(hf_config)
hf_pool.load_state_dict(state, strict=True)

my_output = my_pool(input_ids = input_ids, attention_mask=mask, key=key)
hf_output = hf_pool(input_ids = input_ids_torch, attention_mask=mask_torch, return_dict=False)

check(my_output[0].array, hf_output[0].detach(), ppp=True)

dict_keys(['roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.1.attention.self.query.weight', 'roberta.encoder.layer.2.attention.self.query.weight', 'roberta.encoder.layer.3.attention.self.query.weight', 'roberta.encoder.layer.4.attention.self.query.weight', 'roberta.encoder.layer.5.attention.self.query.weight', 'roberta.encoder.layer.6.attention.self.query.weight', 'roberta.encoder.layer.7.attention.self.query.weight', 'roberta.encoder.layer.8.attention.self.query.weight', 'roberta.encoder.layer.9.attention.self.query.weight', 'roberta.encoder.layer.10.attention.self.query.weight', 'roberta.encoder.layer.11.attention.self.query.weight', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.1.attention.self.query.bias', 'roberta.encoder.layer.2.attention.self.query.bias', 'roberta.encoder.layer.3.attention.self.query.bias', 'roberta.encoder.layer.4.attention.self.query.bias', 'roberta.encoder.layer.5.attention.self.query.bias', 'roberta.

RuntimeError: Error(s) in loading state_dict for RobertaForMaskedLM:
	Missing key(s) in state_dict: "lm_head.bias". 